In [2]:
# traffic_light_detection.py
import cv2
from ultralytics import YOLO
import os
from screeninfo import get_monitors

In [3]:
# Load YOLO model for traffic light detection
model = YOLO('yolov8n.pt')  # Replace with appropriate YOLO model path

In [4]:
# Load the video
#video_path = r"E:\Education\Projects\Machine Learning\Computer Vision\MSD Models\Resources\Videos\5402446-hd_1920_1080_24fps.mp4"
#video_path = r"E:\Education\Projects\Machine Learning\Computer Vision\MSD Models\Resources\Videos\5514514-uhd_3840_2160_24fps.mp4"
video_path = r"E:\Education\Projects\Machine Learning\Computer Vision\MSD Models\Resources\Videos\6272904-uhd_2160_3840_24fps.mp4"
output_dir = r"E:\Education\Projects\Machine Learning\Computer Vision\MSD Models\Resources\Images\Saved Frames"
# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [5]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video file!")
    exit()

In [6]:
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = 0
saved_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or cannot access the video.")
        break

    frame_count += 1
    # Process every frame or skip to reduce processing
    if frame_count % int(fps) != 0:  # Process one frame per second
        continue

    # YOLO detection
    results = model(frame)  # Run the YOLO model on the current frame
    
    # Access the first result in the list
    result = results[0]
    
    # Iterate through detected boxes
    for box in result.boxes:
        class_id = int(box.cls)  # Object class ID
        if class_id == 9:  # Assuming class 9 corresponds to traffic lights
            # Extract bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            traffic_light_roi = frame[y1:y2, x1:x2]  # Crop the traffic light region
    
            # Save the frame with a timestamp or unique name
            save_path = f"{output_dir}/traffic_light_frame{frame_count}.png"
            cv2.imwrite(save_path, traffic_light_roi)
            print(f"Saved ROI: {save_path}")
            saved_count += 1
            break  # Avoid multiple saves for the same frame
            # Resize frame to fit screen size

print(f"Total frames saved: {saved_count}")
cap.release()
cv2.destroyAllWindows()


0: 640x384 2 persons, 2 traffic lights, 1 bench, 178.5ms
Speed: 6.6ms preprocess, 178.5ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 384)
Saved ROI: E:\Education\Projects\Machine Learning\Computer Vision\MSD Models\Resources\Images\Saved Frames/traffic_light_frame23.png

0: 640x384 1 person, 3 traffic lights, 123.5ms
Speed: 4.5ms preprocess, 123.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
Saved ROI: E:\Education\Projects\Machine Learning\Computer Vision\MSD Models\Resources\Images\Saved Frames/traffic_light_frame46.png

0: 640x384 1 person, 2 traffic lights, 107.5ms
Speed: 3.1ms preprocess, 107.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)
Saved ROI: E:\Education\Projects\Machine Learning\Computer Vision\MSD Models\Resources\Images\Saved Frames/traffic_light_frame69.png

0: 640x384 1 person, 2 traffic lights, 109.1ms
Speed: 4.5ms preprocess, 109.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Saved R